In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick
# import scipy.optimize as opt
# import scipy.stats as stat
# from mpl_toolkits.mplot3d import Axes3D
# from operator import itemgetter
# import random
import numpy as np
import numpy.ma as ma
# import numpy.linalg as la
# from mpl_toolkits.mplot3d import Axes3D

# pi = np.pi
# sin = np.sin
# cos = np.cos

In [2]:
import numpy as np
from MSD_utils import get_data_pups, build_time_array, return_average, avg_all
from MSD_utils import SD_all, return_SD, range_and_ticks, choose_y_axis_params, data_prep_for_plotting_pups

In [3]:
path = "./{genotype}/geoM2xy_{sample_name}.csv"
frames = 60
SD_frames = [1, 7, 14, 15]
conversion = (0.5, 1.93, 1)#(0.3, 3.95, 1)
to_frame = 16

parameters = {}
parameters["channels"] = ["RED"]
parameters["genotypes"] = ["WT"]
parameters["pups"] = ["P1", "P2", "P3"]
parameters["surface functionalities"] = ["PEG"]
parameters["slices"] = ["S1", "S2", "S3"]
parameters["regions"] = ["cortex", "mid"]
parameters["replicates"] = [1, 2, 3, 4, 5]

base_name = "WT"

In [5]:
data, avg_over_slices, avg_over_pups, names_with_replicates, time, time_SD, average_over_slices, average_over_pups,\
    all_SD_over_slices, all_SD_over_pups = data_prep_for_plotting_pups(path, frames, SD_frames, conversion, to_frame, parameters)

In [ ]:
def graph_single_variable(all_avg, all_SD, time, time_SD, SD_frames, in_name1, in_name2, to_frame=15, line_colors=['g', 'r', 'b', 'c', 'm', 'k'], line_kind='-',
                         x_manual=False, y_range=10, ticks_y=2, dec_y=0, x_range=5, ticks_x=1, dec_x=0, label_size=95, legend_size=40, tick_size=50,
                         line_width=10, fig_size=(20,18), modify_labels=False, label_identifier="agarose_", base_name="KO"):
    """
    A handy plotting function to examine individual datasets within the larger dataset.
    
    Inputs:
    all_avg: dictionary of numpy arrays.  A subset of this data will be plotted
        based on the parameters in_name1 and in_name2. Keys contain the name of
        the dataset.
    all_SD: dictionary of numpy arrays.  Mirrors the all_avg dataset.
    time: numpy array.  Assumes time array is the same between datasets
        contained in all_avg. 
    time_SD: numpy array.  Contains timepoints at which SDs are desired for the
        plots.
    SD_frames: numpy array.  Contains frames at which SDs are desired for the
        plots.
    in_name1: string.  in_name1 should be found within the keys in all_avg of
        all datasets to be plotted e.g. if I want to plot all RED datasets, the
        I could use in_name1=RED.
    in_name2: similar criteria to in_name1.  If I want to narrow the data to be
        plotted to PEG datasets, then, in_name2 could be "_PEG" (if you only use
        PEG, it will plot all nPEG and PEG datasets, due to my chosen
        nomenclature. Just be aware of your naming system).
    
    to_frame: integer, limits data to be graphed to the range [0, to_frame].
    line_colors: list of strings.  Contains colors desired for plot.
    line_kind: string.  Contains a single string for the desired line format
        of all lines in graph.
    x_manual: True/False.  If True, the user must define range, ticks, and
        decimals for both x and y.
    y_range: y range of the plot.
    ticks_y: interval size on y axis.
    dec_y: number of decimals displayed on y axis.
    x_range" x range of the plot.
    ticks_x: intervals size on x axis.
    dec_x: number of decimals displayed on the x axis.
    label_size: Size of labels on x and y axes.
    legend_size: size of text in legend.
    tick_size: size of text of ticks.
    line_width: width of lines on plot.
    fig_size: x,y size of plot e.g. (20, 18).
    modify_labels: True/False.  If True, the user must give a label_identifier.
    label_identifier: string.  User puts in a string that is contained in all keys
        of all_avg.  Labels of data will now be whatever follows label_identifier
        e.g. if a key contains "agarose_S1" and I use "agarose_", the legend
        will read "S1".
    
    To do:
    I need to make my labelling more flexible eventually.  The troubl is correctly
    lining up my labels with the data in all_avg.
    I also need to make it possible to plot in a desired order e.g. 0.1x, 1x, 10x.
    """
    
    to_graph = {}
    line_type = {}
    counter = 0
    labels = {}
    
    if x_manual == False:
        y_range, ticks_y, dec_y = choose_y_axis_params(all_avg, in_name1, in_name2, to_frame)
        x_range, ticks_x, dec_x = range_and_ticks(time, to_frame)
    else:
        y_range, ticks_y, dec_y = y_range, ticks_y, dec_y
        x_range, ticks_x, dec_x = x_range, ticks_x, dec_x

    filename = base_name + "_" + in_name1 + "_" + in_name2 + ".png"

    # Creates figure
    fig = plt.figure(figsize=fig_size, dpi=80)
    ax = fig.add_subplot(111)

    for keys in all_avg:
        if in_name1 in keys and in_name2 in keys:
            to_graph[counter] = keys
            counter = counter + 1

    for keys in to_graph:
        if modify_labels==True:
            labels[keys] = to_graph[keys].split(label_identifier, 1)[1]
        else:
            labels[keys] = to_graph[keys]
        line_type[keys] = line_colors[keys]+line_kind  
        ax.plot(time[0:to_frame], all_avg[to_graph[keys]][0:to_frame], line_type[keys], linewidth=line_width, label=labels[keys])
        ax.errorbar(time_SD, all_avg[to_graph[keys]][SD_frames], all_SD[to_graph[keys]], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color=line_colors[keys])

    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(tick_size)

    xmajor_ticks = np.arange(0, x_range+0.0001, ticks_x)
    ymajor_ticks = np.arange(0, y_range+0.0001, ticks_y)

    ax.set_xticks(xmajor_ticks)
    plt.xticks(rotation=-30)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(tick_size)
    ax.set_xlabel('Time (s)', fontsize=label_size)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=label_size)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.02, 0.75), prop={'size': legend_size})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec_x)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec_y)))


    # plt.yscale('log')
    # plt.xscale('log')
    plt.gca().set_xlim([0, x_range+0.0001])
    plt.gca().set_ylim([0, y_range+0.0001])

    # Save your figure
    plt.savefig('{}'.format(filename), bbox_inches='tight')
    return y_range, ticks_y, dec_y, x_range, ticks_x, dec_x

In [ ]:
y_range, ticks_y, dec_y, x_range, ticks_x, dec_x = 20, 2.5, 1, 5, 1, 1

In [ ]:
# #Put dataset with largest y range here.

# in_name1 = "YG"
# in_name2 = "YG"

# y_range, ticks_y, dec_y, x_range, ticks_x, dec_x = graph_single_variable(average_over_pups, all_SD_over_pups, time, time_SD, SD_frames, in_name1, in_name2, to_frame=to_frame,
#                       modify_labels=True, label_identifier="PEG_", base_name=base_name)
# plt.show()

In [ ]:
in_name1 = "RED"
in_name2 = "RED"

graph_single_variable(average_over_pups, all_SD_over_pups, time, time_SD, SD_frames, in_name1, in_name2, to_frame=to_frame, y_range=y_range,
                      ticks_y=ticks_y, dec_y=dec_y, x_range=x_range, ticks_x=ticks_x, dec_x=dec_x,
                      modify_labels=True, label_identifier="PEG_", x_manual=True, base_name=base_name)
plt.show()

In [ ]:
in_name1 = ["cortex", "mid"]
in_name2 = ["RED", "YG"]

for one in in_name1:
    for two in in_name2:

        graph_single_variable(average_over_slices, all_SD_over_slices, time, time_SD, SD_frames, one, two, to_frame=to_frame, y_range=y_range,
                              ticks_y=ticks_y, dec_y=dec_y, x_range=x_range, ticks_x=ticks_x, dec_x=dec_x,
                              modify_labels=True, label_identifier="PEG_", x_manual=True, base_name=base_name)
#plt.show()